In [38]:
import cv2
from knn_process import knn_process_df_image

In [39]:
image_path = '../../dataset_test/Banana/1.jpg'
knn_process_df_image(image_path=image_path)


,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_862760,feat_862761,feat_862762,feat_862763,feat_862764,feat_862765,feat_862766,feat_862767,feat_862768,feat_862769
0,255,255,255,255,255,255,255,255,255,255,...,0.029786,0.016791,0.032429,0.042543,0.076006,0.024441,0.038013,0.661818,0.054466,"{'contrast': 306.49704204705466, 'dissimilarit..."


In [40]:
image_process = cv2.imread(image_path)
knn_process_df_image(image_process=image_process)

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_862760,feat_862761,feat_862762,feat_862763,feat_862764,feat_862765,feat_862766,feat_862767,feat_862768,feat_862769
0,255,255,255,255,255,255,255,255,255,255,...,0.029786,0.016791,0.032429,0.042543,0.076006,0.024441,0.038013,0.661818,0.054466,"{'contrast': 306.49704204705466, 'dissimilarit..."
